In [1]:
SCENARIOS = [
    r"C:\Users\darthoma\Documents\CEA-Projects\reference-case-open.3.4.0\baseline",
    r"C:\Users\darthoma\Documents\CEA-Projects\reference-case-open.3.5.0a2\baseline",
    r"C:\Users\darthoma\Documents\CEA-Projects\reference-case-open.3.5.0a3\baseline",
    r"C:\Users\darthoma\Documents\CEA-Projects\reference-case-open.3.5.0a3-new-environment\baseline"
]
SCENARIO_NAMES = ["3.4.0", "3.5.0a2", "3.5.0a3", "3.5.0a4"]

In [2]:
import cea.inputlocator
locators = [cea.inputlocator.InputLocator(s) for s in SCENARIOS]
locator = locators[0]

In [3]:
import cea.schemas
schemas = cea.schemas.schemas(plugins=[])

In [4]:
import yaml
schemas_yml = r"C:\Users\darthoma\Documents\GitHub\CityEnergyAnalyst\cea\schemas.yml"
with open(schemas_yml, "r") as fp:
    schemas = yaml.safe_load(fp)

In [5]:
SKIP = {
    "get_optimization_building_scale_heating_capacity", # ??
    "get_optimization_district_scale_heating_capacity", # ??
    "get_optimization_generation_building_scale_performance", # debug only
    "get_optimization_generation_district_scale_performance", # debug only
    "get_optimization_generation_total_performance", # debug only
    "get_optimization_network_results_summary", # barcode...
    "get_optimization_substations_results_file", # barcode
    "get_optimization_substations_total_file"
}

In [6]:
import warnings
warnings.simplefilter("ignore")

# figure out input locators to use...
# schemas_yml = r"C:\Users\darthoma\Documents\GitHub\CityEnergyAnalyst\cea\schemas.yml"
# with open(schemas_yml, "r") as fp:
#    schemas = yaml.safe_load(fp)

# these are the files we're going to check
default_args = {
    "building": "B1000",
    "network_type": "DH",
    "network_name": "",
    "panel_type": "ET",
    "use": "RESTAURANT",
    "format": "csv",
    "generation": 2,
    "representative_week": False,
    "ind_num": 0,
    "gen_num": 2,
    "configuration": "AHU_ARU_SCU",
}

for lm in schemas.keys():
    if lm in SKIP:
        continue
        
    if not schemas[lm]["file_type"] in ("csv", "shp", "dbf"):
        # ignore excel files etc. for now...
        continue        
    
    print(f"Checking lm={lm}")
    
    sio = getattr(locator, lm)
    method = sio.original_function
    
    
    try:
        args = {p: default_args[p] for p in schemas[lm]["parameters"]}
    except KeyError:
        print(f"Failed on lm={lm}")
        raise
        
    try:
        sio.read(**args)
    except FileNotFoundError:
        print(f"Failed on lm={lm}")
        raise   

Checking lm=PVT_metadata_results
Checking lm=PVT_results
Checking lm=PVT_total_buildings
Checking lm=PVT_totals
Checking lm=PV_metadata_results
Checking lm=PV_results
Checking lm=PV_total_buildings
Checking lm=PV_totals
Checking lm=SC_metadata_results
Checking lm=SC_results
Checking lm=SC_total_buildings
Checking lm=SC_totals
Checking lm=get_building_air_conditioning
Checking lm=get_building_architecture
Checking lm=get_building_comfort
Checking lm=get_building_internal
Checking lm=get_building_supply
Checking lm=get_building_typology
Checking lm=get_building_weekly_schedules
Checking lm=get_costs_operation_file
Checking lm=get_demand_results_file
Checking lm=get_geothermal_potential
Checking lm=get_lca_embodied
Checking lm=get_lca_operation
Checking lm=get_multi_criteria_analysis
Checking lm=get_network_energy_pumping_requirements_file
Checking lm=get_network_layout_edges_shapefile
Checking lm=get_network_layout_nodes_shapefile
Checking lm=get_network_linear_pressure_drop_edges
Checki

In [7]:
# Figure out the files that differ...

for lm in schemas.keys():
    if lm in SKIP:
        continue
        
    if not schemas[lm]["file_type"] in ("csv", "shp", "dbf"):
        # ignore excel files etc. for now...
        continue        
    
    print(f"Checking lm={lm}")
    
    sios = [getattr(loc, lm) for loc in locators]
    sio = getattr(locator, lm)
    method = sio.original_function
    
    
    try:
        args = {p: default_args[p] for p in schemas[lm]["parameters"]}
    except KeyError:
        print(f"Failed on lm={lm}")
        raise
        
    try:
        sio.read(**args)
    except FileNotFoundError:
        print(f"Failed on lm={lm}")
        raise   

Checking lm=PVT_metadata_results
Checking lm=PVT_results
Checking lm=PVT_total_buildings
Checking lm=PVT_totals
Checking lm=PV_metadata_results
Checking lm=PV_results
Checking lm=PV_total_buildings
Checking lm=PV_totals
Checking lm=SC_metadata_results
Checking lm=SC_results
Checking lm=SC_total_buildings
Checking lm=SC_totals
Checking lm=get_building_air_conditioning
Checking lm=get_building_architecture
Checking lm=get_building_comfort
Checking lm=get_building_internal
Checking lm=get_building_supply
Checking lm=get_building_typology
Checking lm=get_building_weekly_schedules
Checking lm=get_costs_operation_file
Checking lm=get_demand_results_file
Checking lm=get_geothermal_potential
Checking lm=get_lca_embodied
Checking lm=get_lca_operation
Checking lm=get_multi_criteria_analysis
Checking lm=get_network_energy_pumping_requirements_file
Checking lm=get_network_layout_edges_shapefile
Checking lm=get_network_layout_nodes_shapefile
Checking lm=get_network_linear_pressure_drop_edges
Checki

In [8]:
def float_and_int_columns(schemas, lm):
    result = []
    for col in schemas[lm]["schema"]["columns"]:
        col_type = schemas[lm]["schema"]["columns"][col]["type"]
        if col_type in {"float", "int"}:
            result.append(col)
    return result

In [9]:
# simple test with demand:
# %matplotlib widget
from functools import reduce
import cufflinks
import matplotlib.pyplot as plt
import pandas as pd
import os

outpath = r"c:\Users\darthoma\Documents\CEA-Projects\compare"

lm = "get_total_demand"
sios = [getattr(loc, lm) for loc in locators]
args = {p: default_args[p] for p in schemas[lm]["parameters"]}
print(f"{lm} parameters: {args}")
dfs = [sio.read(**args) for sio in sios]
columns = float_and_int_columns(schemas, lm)

figfolder = f"{lm}"
figabsfolder = os.path.join(outpath, figfolder)    
if not os.path.exists(figabsfolder):
    os.mkdir(figabsfolder)

for column in columns:
    title=f"{lm}.{column}"
    figname = f"{column}.png"            
    df = pd.DataFrame([df[column] for df in dfs], index=SCENARIO_NAMES)    
    print(f"Saving {figname}")
    df.plot(title=title, legend=None)
    plt.savefig(os.path.join(figabsfolder, figname))
    plt.close()

get_total_demand parameters: {'format': 'csv'}
Saving Af_m2.png
Saving Aocc_m2.png
Saving Aroof_m2.png
Saving COAL_hs0_kW.png
Saving COAL_hs_MWhyr.png
Saving COAL_ww0_kW.png
Saving COAL_ww_MWhyr.png
Saving DC_cdata0_kW.png
Saving DC_cdata_MWhyr.png
Saving DC_cre0_kW.png
Saving DC_cre_MWhyr.png
Saving DC_cs0_kW.png
Saving DC_cs_MWhyr.png
Saving DH_hs0_kW.png
Saving DH_hs_MWhyr.png
Saving DH_ww0_kW.png
Saving DH_ww_MWhyr.png
Saving E_cdata0_kW.png
Saving E_cdata_MWhyr.png
Saving E_cre0_kW.png
Saving E_cre_MWhyr.png
Saving E_cs0_kW.png
Saving E_cs_MWhyr.png
Saving E_hs0_kW.png
Saving E_hs_MWhyr.png
Saving E_sys0_kW.png
Saving E_sys_MWhyr.png
Saving E_ww0_kW.png
Saving E_ww_MWhyr.png
Saving Ea0_kW.png
Saving Ea_MWhyr.png
Saving Eal0_kW.png
Saving Eal_MWhyr.png
Saving Eaux0_kW.png
Saving Eaux_MWhyr.png
Saving Edata0_kW.png
Saving Edata_MWhyr.png
Saving El0_kW.png
Saving El_MWhyr.png
Saving Epro0_kW.png
Saving Epro_MWhyr.png
Saving GFA_m2.png
Saving GRID0_kW.png
Saving GRID_MWhyr.png
Saving 

In [10]:
pd.DataFrame([df["Af_m2"] for df in dfs], index=SCENARIO_NAMES)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
3.4.0,8872.306,3283.739,3060.552,2937.421,3060.674,4990.466,4625.525,4193.764,5095.311,1577.16,1090.678,2190.412,4726.611,2109.126,11408.669
3.5.0a2,8872.306,3283.739,3060.552,2937.421,3060.674,4990.466,4625.525,4193.764,5095.311,1577.16,1090.678,2190.412,4726.611,2109.126,11408.669
3.5.0a3,8872.306,3283.739,3060.552,2937.421,3060.674,4990.466,4625.525,4193.764,5095.311,1577.16,1090.678,2190.412,4726.611,2109.126,11408.669
3.5.0a4,8872.306,3283.739,3060.552,2937.421,3060.674,4990.466,4625.525,4193.764,5095.311,1577.16,1090.678,2190.412,4726.611,2109.126,11408.669


# check these more thoroughly:

- get_optimization_substations_results_file